In [4]:
# importing the libraries
import pandas as pd
import artm
import math
from tqdm import tqdm

In [5]:
# uploading data & picking only text part
df = pd.read_csv(r'/Users/apple/Documents/BDML /НИР/train.csv')
dataset1 = df['text']

# creating the function for transformation to vowpal_wabbit format


def df_to_vw_regression(df, filepath='in.txt', columns=None, target=None, namespace='namespace'):
    if columns is None:
        columns = df.columns.tolist()
    columns.remove(target)

    with open(filepath, 'w') as f:
        for _, row in tqdm(df.iterrows()):
            if namespace:
                f.write('{0} |{1} '.format(row[target], namespace))
            else:
                f.write('{0} | '.format(row[target]))
            last_feature = row.index.values[-1]
            for idx, val in row.iteritems():
                if idx not in columns:
                    continue
                if isinstance(val, str):
                    f.write('{0}'.format(val.replace(' ', '_').replace(':', '_')))
                elif isinstance(val, float) or isinstance(val, int):
                    if not math.isnan(val):
                        f.write('{0}:{1}'.format(idx.replace(' ', '_').replace(':', '_'), val))
                    else:
                        continue
                else:
                    f.write('{0}'.format(val.replace(' ', '_').replace(':', '_')))
                if idx != last_feature:
                    f.write(' ')
            f.write('\n')


def df_to_vw_classification(
        df,
        filepath='mc.txt',
        columns=None,
        target=None,
        tag=None,
        namespace='n'
):
    if columns is None:
        columns = df.columns.tolist()
    columns.remove(target)
    columns.remove(tag)

    with open(filepath, 'w') as f:
        for _, row in tqdm(df.iterrows()):
            if namespace:
                f.write(f"{row[target]} \'{row[tag]} |{namespace} ")
            else:
                f.write(f"{row[target]} \'{row[tag]} | ")
            last_feature = columns[-1]
            for idx, val in row.iteritems():
                if idx not in columns:
                    continue
                if isinstance(val, str):
                    f.write(f"{val.replace(' ', '_').replace(':', '_')}")
                elif isinstance(val, float) or isinstance(val, int):
                    if not math.isnan(val):
                        f.write(f"{idx.replace(' ', '_').replace(':', '_')}:{val}")
                    else:
                        continue
                else:
                    f.write(f"{val.replace(' ', '_').replace(':', '_')}")
                if idx != last_feature:
                    f.write(' ')
            f.write('\n')

In [3]:
# changing the type of data created
vw = df_to_vw_regression(df, filepath='data.txt', target='text')
# vw = df_to_vw_classification(df, filepath='data.txt', target='target')

73987it [00:22, 3327.24it/s]


In [4]:
# batching data for applying it to our model


batch_vectorizer = artm.BatchVectorizer(data_path='data.txt',
                                        data_format='vowpal_wabbit',
                                        collection_name='vw',
                                        target_folder='koselniy_batches')

# LDA (BigARTM package) Model

In [5]:
# setting up lda parameters


lda = artm.LDA(num_topics=10,
               alpha=0.01, beta=0.001,
               cache_theta=True,
               num_document_passes=2,
               dictionary=batch_vectorizer.dictionary)
#Phi is the ‘parts-versus-topics’ matrix, and theta is the ‘composites-versus-topics’ matrix
lda.fit_offline(batch_vectorizer=batch_vectorizer,
                num_collection_passes=10)

#checking up the parametrs of the matricies
lda.sparsity_phi_last_value
lda.sparsity_theta_last_value
lda.perplexity_last_value

1.257523775100708

In [6]:
#top tokens for each class of all the clusters we've got
top_tokens = lda.get_top_tokens(num_tokens=200)
for i, token_list in enumerate(top_tokens):
    print('Topic #{0}: {1}'.format(i, token_list))

d = pd.DataFrame(top_tokens)
top_tokens_vec= d.apply(lambda x: ' '.join(x), axis=1)

Topic #0: ['человека', 'жизни', 'лет', 'года', 'сегодня', 'цена', 'год', 'людей', 'работа', 'других', 'жизнь', 'sex', 'big', 'дня', 'день', 'секс', 'работы', 'любовь', 'времени', 'фото', 'каждый', 'стороны', 'сколько', 'состояние', 'образом', 'несколько', 'друга', 'первый', 'самый', 'мире', 'любви', 'лишь', 'tits', 'amp', 'руб', 'какие', 'хотя', 'часть', 'рядом', 'деньги', 'хочу', 'отдам', 'снова', 'должен', 'новый', 'состоянии', 'кроме', 'рублей', 'делать', 'самые', 'июля', 'города', 'порно', 'полностью', 'anal', 'сентября', 'размер', 'дело', 'прямо', 'завтра', 'места', 'знает', 'момент', 'мама', 'поэтому', 'жить', 'другие', 'говорит', 'любит', 'проблемы', 'имя', 'знаю', 'месте', 'самых', 'работе', 'дела', 'помощью', 'сердце', 'июня', 'вместе', 'ass', 'дома', 'вашего', 'счастье', 'сша', 'цвет', 'мая', 'porn', 'думаю', 'пару', 'марта', 'необходимо', 'россии', 'сил', 'качестве', 'кажется', 'почему', 'девушка', 'тел', 'году', 'друзья', 'работать', 'силы', 'родился', 'жена', 'хотите', 'на

# ARTM (BigARTM package) Model

In [7]:
#creating the batches for the second model and setting up bigartm dictionary
batch_vectorizer2 = artm.BatchVectorizer(data_path='data.txt',
                                         data_format='vowpal_wabbit',
                                         collection_name='vw',
                                         target_folder='koselniy_batches')
dictionary = batch_vectorizer2.dictionary

In [8]:
topic_names = ['topic_{}'.format(i) for i in range(15)]
#inial objects cration
model_plsa = artm.ARTM(topic_names=topic_names, cache_theta=True,
                       scores=[artm.PerplexityScore(name='PerplexityScore',
                                                    dictionary=dictionary)])
model_artm = artm.ARTM(topic_names=topic_names, cache_theta=True,
                       scores=[artm.PerplexityScore(name='PerplexityScore',
                                                    dictionary=dictionary)],
                       regularizers=[artm.SmoothSparseThetaRegularizer(name='SparseTheta',
                                                                       tau=-0.15)])

In [9]:
#adding some scores for our future model
model_plsa.scores.add(artm.SparsityPhiScore(name='SparsityPhiScore'))
model_plsa.scores.add(artm.SparsityThetaScore(name='SparsityThetaScore'))
model_plsa.scores.add(artm.TopicKernelScore(name='TopicKernelScore',probability_mass_threshold=0.3))
model_plsa.scores.add(artm.TopTokensScore(name='Top_words', num_tokens=15, class_id='text'))
model_artm.scores.add(artm.SparsityPhiScore(name='SparsityPhiScore'))
model_artm.scores.add(artm.SparsityThetaScore(name='SparsityThetaScore'))
model_artm.scores.add(artm.TopicKernelScore(name='TopicKernelScore',probability_mass_threshold=0.3))
model_artm.scores.add(artm.TopTokensScore(name='Top_words', num_tokens=15, class_id='text'))
model_plsa.scores.add(artm.TopTokensScore(name='TopTokensScore', num_tokens=200))
model_artm.scores.add(artm.TopTokensScore(name='TopTokensScore', num_tokens=200))

#regulizers
model_artm.regularizers.add(artm.SmoothSparsePhiRegularizer(name='SparsePhi', tau=-0.1))
model_artm.regularizers.add(artm.DecorrelatorPhiRegularizer(name='DecorrelatorPhi', tau=0.01))

model_plsa.num_document_passes = 1
model_artm.num_document_passes = 1

#initializing the model we've set up
model_plsa.initialize(dictionary=dictionary)
model_artm.initialize(dictionary=dictionary)

In [10]:
#fitting the model
#model_plsa.fit_offline(batch_vectorizer=batch_vectorizer2, num_collection_passes=15)
model_artm.fit_offline(batch_vectorizer=batch_vectorizer2, num_collection_passes=10)

In [ ]:
#pickling te new model

In [12]:
#checking out the sparsity of our matricies
#print(model_artm.score_tracker["SparsityPhiScore"].last_value)
#print(model_artm.score_tracker["SparsityThetaScore"].last_value)

print(model_artm.score_tracker["SparsityPhiScore"].last_value)
print(model_artm.score_tracker["SparsityThetaScore"].last_value)

#checking out topic kernel score
print(model_artm.score_tracker["TopicKernelScore"])

#print(model_plsa.score_tracker["TopicKernelScore"])

0.7198261022567749
0.0


In [13]:
#checking up the top words
#print(model_artm.score_tracker["Top_words"])
print(model_artm.score_tracker["Top_words"])
#print(model_artm.)

In [14]:
for topic_name in model_artm.topic_names:
    print(topic_name + ': '),
    print(model_artm.score_tracker['TopTokensScore'].last_tokens[topic_name])

topic_0: 
['человека', 'жизни', 'людей', 'лет', 'жизнь', 'сегодня', 'любви', 'года', 'руб', 'любовь', 'рядом', 'других', 'любить', 'год', 'сколько', 'лишь', 'день', 'времени', 'цена', 'работа', 'каждый', 'самый', 'сердце', 'снова', 'дня', 'значит', 'друга', 'работы', 'счастье', 'должен', 'какие', 'хотя', 'хочешь', 'фото', 'жить', 'стороны', 'первый', 'самые', 'девушка', 'деньги', 'несколько', 'поэтому', 'проблемы', 'делать', 'дело', 'слова', 'мире', 'любит', 'знает', 'кроме', 'завтра', 'дела', 'чувства', 'твои', 'никто', 'момент', 'другие', 'состояние', 'силы', 'хочу', 'хочется', 'самых', 'слишком', 'кажется', 'почему', 'работу', 'месте', 'мысли', 'твоя', 'сказать', 'мама', 'города', 'достаточно', 'полностью', 'имя', 'говорит', 'необходимо', 'вместе', 'часть', 'говорить', 'рублей', 'дома', 'новый', 'важно', 'смотреть', 'помощью', 'твоей', 'отношения', 'знаю', 'мая', 'меньше', 'хочет', 'бог', 'например', 'долго', 'состоянии', 'пор', 'постоянно', 'прямо', 'места', 'сил', 'отдам', 'июля',

['жизнь', 'люди', 'любовь', 'сегодня', 'день', 'года', 'жизни', 'хочется', 'друг', 'слова', 'работы', 'хочешь', 'цена', 'людям', 'новый', 'именно', 'году', 'стать', 'продам', 'никто', 'хочет', 'детей', 'человека', 'жить', 'самое', 'делать', 'людей', 'руб', 'твои', 'рублей', 'дом', 'например', 'фото', 'постоянно', 'деньги', 'слишком', 'пожалуйста', 'несколько', 'почему', 'россии', 'какие', 'мужчина', 'является', 'сделать', 'вопрос', 'внутри', 'проблемы', 'сразу', 'быстро', 'мысли', 'равно', 'лет', 'глаза', 'сердце', 'женщина', 'твоя', 'любой', 'обычно', 'других', 'дети', 'хочу', 'мира', 'раньше', 'любят', 'плохо', 'говорят', 'никому', 'снова', 'сказал', 'дома', 'получить', 'отношения', 'подскажите', 'год', 'пару', 'минут', 'людьми', 'должна', 'хотя', 'сентября', 'вещи', 'нужна', 'должен', 'руки', 'души', 'работу', 'важно', 'правильно', 'момент', 'особенно', 'мало', 'мама', 'некоторые', 'июня', 'дней', 'здравствуйте', 'писать', 'пути', 'утром', 'состояние', 'две', 'возможность', 'душе', 

['жизни', 'каждый', 'люди', 'лишь', 'лет', 'день', 'любовь', 'люблю', 'сердце', 'друга', 'сделать', 'поэтому', 'значит', 'друг', 'глаза', 'важно', 'цена', 'рублей', 'хочу', 'хочешь', 'продам', 'времени', 'жизнь', 'сколько', 'друзья', 'знаешь', 'руки', 'место', 'новый', 'немного', 'россии', 'женщины', 'легко', 'никто', 'почему', 'любишь', 'говорит', 'пишите', 'любит', 'часто', 'знать', 'человека', 'знаю', 'достаточно', 'сказал', 'долго', 'дом', 'минут', 'хочет', 'слишком', 'момент', 'сразу', 'самые', 'бывает', 'понять', 'говорят', 'должны', 'сильно', 'людей', 'твоя', 'женщина', 'возможно', 'найти', 'мира', 'любви', 'мая', 'несколько', 'первый', 'девушка', 'состояние', 'утром', 'жить', 'большой', 'ночь', 'друзей', 'однажды', 'мужчины', 'равно', 'сказать', 'вопрос', 'мире', 'пожалуйста', 'месте', 'пару', 'работы', 'хотите', 'свете', 'места', 'должен', 'имя', 'твоё', 'купить', 'душу', 'сегодня', 'особенно', 'слова', 'дня', 'ноги', 'обязательно', 'размер', 'вашей', 'раньше', 'должна', 'любо

In [15]:
artmTokens = model_artm.score_tracker['TopTokensScore'].last_tokens
artmTokens = pd.DataFrame(artmTokens)
artmToks= artmTokens.apply(lambda x: ' '.join(x), axis=0)
artmToks

topic_0     человека жизни людей лет жизнь сегодня любви г...
topic_1     день сегодня лишь стоит люди жизни лет мир люб...
topic_2     день жизни люди людей каждый сколько лет любов...
topic_3     человека лет каждый хочу рядом людей сколько ф...
topic_4     жизнь жизни день человека сегодня рядом значит...
topic_5     жизнь стоит лишь жизни люди хочу года день дру...
topic_6     жизнь люди любовь сегодня день года жизни хоче...
topic_7     день жизнь людей люди лишь почему жизни хочу л...
topic_8     жизнь руб человека лет день года стоит мир люб...
topic_9     жизни люди человека людей почему поэтому девуш...
topic_10    люди сегодня года людей любовь лет стоит челов...
topic_11    жизни каждый люди лишь лет день любовь люблю с...
topic_12    люди жизни года сегодня лишь людей руб хочу жи...
topic_13    день жизни каждый лишь мир людям нужен сделать...
topic_14    жизнь людей люди сегодня день человека любовь ...
dtype: object

# KMeans model

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords

/Users/apple/anaconda3/lib/python3.7/site-packages/nltk/decorators.py:70: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  formatvalue=lambda value: "")[1:-1]


In [81]:
# retrieving text: setting up stop words, making Tf-Idf
words = stopwords.words('russian')
vectorizer = TfidfVectorizer(stop_words=words)
X = vectorizer.fit_transform(dataset1)

In [82]:
# counting KMeans for 13 clusters and fitting vectorized data in to the model
true_k = 18
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=10, n_init=1)
model.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=10,
       n_clusters=18, n_init=1, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [83]:
#figuring out centroids for our data
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
centers = model.cluster_centers_

In [84]:
for i in range(true_k):
    print('Cluster %d:' % i),
    for ind in order_centroids[i, :300]:
        print(' %s' % terms[ind])

Cluster 0:
 волос
 волосы
 кожи
 наращивание
 дизайн
 кожу
 цвет
 ресниц
 ногтей
 маникюр
 макияж
 гель
 кухни
 идея
 лица
 цвета
 кухня
 интерьер
 процедуры
 кожа
 крем
 маска
 ресницы
 ногти
 масла
 прядей
 масло
 маски
 спальня
 бровей
 макияжа
 средство
 сделать
 покрытие
 наращивания
 эффект
 процедура
 выпрямление
 волосами
 гостиной
 коже
 лак
 интерьере
 спальни
 оттенки
 головы
 мастер
 запись
 стоит
 блеск
 гостиная
 день
 средства
 волосам
 использовать
 маску
 руб
 окрашивание
 интерьера
 поэтому
 прически
 сколько
 прическа
 водой
 здравствуйте
 мастера
 комнаты
 шампунь
 минут
 глаз
 девочки
 роста
 оттенок
 кератиновое
 квартиры
 объем
 кожей
 коричневый
 стиле
 брови
 волоса
 помощью
 цена
 нанесите
 наносить
 коррекция
 ванной
 уход
 акция
 губ
 красоты
 делать
 реснички
 процедуру
 пряди
 внимание
 лаком
 несколько
 комната
 необходимо
 мебели
 мебель
 ухода
 затем
 педикюр
 состав
 студии
 глаза
 стрижка
 депиляции
 именно
 дом
 ванная
 окрашивания
 тон
 нанести
 вол

 пакет
 пишите
 состояние
 выберу
 вещей
 обувь
 обменяю
 руки
 хорошем
 шоколадку
 рост
 вкусняшку
 отдаю
 срочно
 куртка
 лет
 джинсы
 размера
 добрые
 детские
 платье
 девочки
 новые
 продукты
 года
 вопросы
 лотку
 туфли
 женские
 котят
 отличном
 размеры
 примерно
 вкусняшки
 сахара
 мес
 брюки
 сапоги
 пачку
 лоток
 порошок
 метро
 осень
 пакета
 предложения
 бесплатно
 пожалуйста
 костюм
 памперсы
 тел
 возраст
 пальто
 новый
 штаны
 новая
 игрушки
 юбка
 месяца
 дома
 платья
 месяцев
 хорошее
 дом
 химию
 репосту
 хорошие
 желательно
 девочка
 одежда
 дачи
 фрукты
 самовынос
 кушают
 комбинезон
 забрать
 ботинки
 киндер
 бронирую
 звоните
 коляску
 кофты
 iphone
 детскую
 мальчик
 сегодня
 завтра
 пачки
 зима
 сроки
 шоколад
 диван
 корм
 детский
 приучены
 презент
 куртку
 стельке
 пакетом
 центр
 две
 рубашка
 здравствуйте
 отличное
 штук
 брони
 бытовую
 кофта
 сообщения
 шорты
 районе
 кроссовки
 одежду
 рабочий
 масло
 пиджак
 отслеживаю
 масла
 кушает
 меняю
 кроватку
 во

 любить
 счастье
 лишь
 почему
 день
 жить
 каждый
 любви
 хочется
 женщина
 значит
 глаза
 знаю
 никто
 мама
 мужчина
 знаешь
 любит
 мир
 девушка
 хочешь
 твои
 друга
 лет
 любишь
 сегодня
 вместе
 равно
 самое
 говорит
 говорят
 слишком
 сказать
 бывает
 сказал
 нужен
 однажды
 слова
 сколько
 боль
 больно
 снова
 женщины
 твоей
 делать
 твоя
 мужчины
 давай
 душу
 важно
 главное
 самый
 любят
 счастья
 парень
 нравится
 людям
 человеку
 ради
 кажется
 дома
 руки
 стоит
 друзья
 душа
 говорить
 нахуй
 сильно
 хочет
 плохо
 душе
 муж
 никому
 чувства
 внутри
 спать
 человеком
 бог
 ждать
 души
 момент
 завтра
 мире
 должен
 друзей
 отношения
 домой
 девушки
 ночь
 знает
 долго
 навсегда
 жена
 понимаешь
 забыть
 чувство
 ненавижу
 сразу
 понять
 ночью
 именно
 мысли
 думаешь
 нужна
 год
 сделать
 никого
 думаю
 твою
 действительно
 мало
 сказала
 часто
 блять
 скучаю
 самая
 каждого
 знать
 утро
 других
 мужик
 твоих
 хотя
 ночи
 девочка
 свете
 папа
 дети
 найти
 видеть
 смотреть
 п

 свадьбе
 свадебного
 свадебные
 фотограф
 день
 гостей
 огромное
 фото
 букет
 платье
 фотографии
 гости
 праздник
 ведущий
 свадебных
 свадебной
 невеста
 платья
 фотосессии
 свадебное
 жениха
 свадебная
 фотосессия
 большое
 ведущего
 восторге
 торжества
 видео
 здравствуйте
 стоимость
 вашей
 оформление
 фотографа
 молодоженов
 праздника
 сколько
 свадеб
 ваши
 гостям
 макияж
 съемки
 стиле
 торт
 проведение
 молодожены
 дорогие
 добрый
 друзья
 съемка
 именно
 года
 жених
 сделать
 супер
 студии
 услуги
 невест
 весело
 конкурсы
 жизни
 любовь
 подарок
 фотосессию
 любви
 церемонии
 букеты
 шоу
 ребята
 дня
 понравилось
 ваших
 августа
 декор
 тамада
 стоит
 работы
 фотографий
 story
 love
 настроение
 свадебную
 торжество
 прическа
 лет
 цветов
 стоить
 цветы
 wedding
 успехов
 вечер
 хотим
 свадебном
 довольны
 счастья
 проведения
 благодарность
 пары
 друг
 даты
 яркие
 цвет
 личку
 ресторан
 приятно
 съемку
 идеи
 самый
 образ
 подскажите
 поэтому
 вместе
 выразить
 заранее
 и

In [85]:
kdf = pd.DataFrame(
    [i,  terms[ind]] 
    for i in range(true_k)
    for ind in order_centroids[i, :30]).groupby([0])[1].transform(lambda x: ' '.join(x)).drop_duplicates()

In [109]:
print(kdf[1:2])

30    день жизни сегодня лет жизнь года люди фото лю...
Name: 1, dtype: object


# RuBETR classifier

In [80]:
import warnings
warnings.filterwarnings('ignore')

In [82]:
from deeppavlov.dataset_iterators.basic_classification_iterator import BasicClassificationDatasetIterator
#iterator = BasicClassificationDatasetIterator(data, seed=42, shuffle=True)
#for batch in iterator.gen_batches(data_type="train", batch_size=13):
#    print(batch)
#    break

In [31]:
#importing more libraries
import math

#downloading te right vocabulaty
from deeppavlov.models.preprocessors.bert_preprocessor import BertPreprocessor
bert_preprocessor = BertPreprocessor(vocab_file="~/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v2/vocab.txt",
                                     do_lower_case=False,
                                     max_seq_length=64)



In [32]:
from deeppavlov.models.bert.bert_classifier import BertClassifierModel
from deeppavlov.metrics.accuracy import sets_accuracy
bert_classifier = BertClassifierModel(
    n_classes=15,
    return_probas=False,
    one_hot_labels=True,
    bert_config_file="~/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v2/bert_config.json",
    pretrained_bert="~/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v2/bert_model.ckpt",
    save_path="ru_bert_model/model",
    load_path="ru_bert_model/model",
    keep_prob=0.5,
    learning_rate=1e-05,
    learning_rate_drop_patience=5,
    learning_rate_drop_div=2.0
)

2019-11-24 22:28:58.622 INFO in 'deeppavlov.models.bert.bert_classifier'['bert_classifier'] at line 98: [initializing model with Bert from /Users/apple/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v2/bert_model.ckpt]


INFO:tensorflow:Restoring parameters from /Users/apple/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v2/bert_model.ckpt


In [33]:
#changing vocabulary format
from deeppavlov.core.data.simple_vocab import SimpleVocabulary
vocab = SimpleVocabulary(save_path="./binary_classes.dict")
#iterator.get_instances(data_type="train")
#vocab.fit(iterator.get_instances(data_type="train")[1])
#list(vocab.items())
#bert_classifier(bert_preprocessor(top_tokens_vec))
#bert_classifier(bert_preprocessor(artmToks))

2019-11-24 22:29:03.172 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 47: No load path is set for SimpleVocabulary in 'infer' mode. Using save path instead


[]

# Turning BERT classifier into pretrained PyTorch

from pytorch_transformers import BertForPreTraining, BertConfig, BertTokenizer, load_tf_weights_in_bert, BertModel

#transformin deeppavlov BERT model into PyTorch pretrained model

def convert_tf_checkpoint_to_pytorch(tf_checkpoint_path, bert_config_file):
    config = BertConfig.from_json_file(bert_config_file)
    model = BertForPreTraining(config)
    model = load_tf_weights_in_bert(model, config, tf_checkpoint_path)

    return model

BERT_MODEL_PATH = "/Users/apple/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v2/"
tf_checkpoint_path = BERT_MODEL_PATH + 'bert_model.ckpt'
bert_config_file = BERT_MODEL_PATH + 'bert_config.json'

downloaded = convert_tf_checkpoint_to_pytorch(tf_checkpoint_path, bert_config_file)
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None, do_lower_case=False)

encoder = downloaded.bert

In [1]:
#working summarizer

#needs customization:
# 1) adding Rus from spacy
# 2) adding multilang bert to bertParent
# 3) changing model and language all over the preproccessing file
from summarizer import Summarizer
model = Summarizer()

In [2]:
body = '''
Как уточнил собеседник агентства, предоставление дополнительного выходного, кроме случаев, прописанных в законе, может повлечь уменьшение заработной платы.

Кроме того, сотрудник вправе попросить выходной 31 декабря, но без сохранения зарплаты, или взять отгул за работу сверхурочно, в выходной либо праздник или как часть ежегодного отпуска.
Роструд также напомнил, что 31 декабря предшествует нерабочему праздничному дню, поэтому его продолжительность сокращается на один час.'''


result = model(body, min_length=6)
full = ''.join(result)
print(full)

Как уточнил собеседник агентства, предоставление дополнительного выходного, кроме случаев, прописанных в законе, может повлечь уменьшение заработной платы. Кроме того, сотрудник вправе попросить выходной 31 декабря, но без сохранения зарплаты, или взять отгул за работу сверхурочно, в выходной либо праздник или как часть ежегодного отпуска.


In [5]:
#working summarizer
text = body

from summa import keywords
lables = keywords.keywords(text, language = 'russian',ratio = 0.02, scores = False)
print(lables)

выходного
выходной


In [18]:
from summa import summarizer
sumar = summarizer.summarize(text)
print(sumar)

# TFIDF Classifier based on LentaRu dataset

In [34]:
# uploading data
lentaRu = pd.read_csv(r'/Users/apple/Downloads/lenta-ru-news.csv')
lentaRuTxt = lentaRu['text']
lentaRu.info()

# importing all libraries needed
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

#scikit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2

#defining classes stemmer and stop words
stemmer = SnowballStemmer('russian')
words = stopwords.words('russian')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/apple/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 1995, in read
    data = self._reader.read(nrows)
  File "pandas/_libs/parsers.pyx", line 899, in pandas._libs.parsers.TextReader.read
  File "pandas/_libs/parsers.pyx", line 914, in pandas._libs.parsers.TextReader._read_low_memory
  File "pandas/_libs/parsers.pyx", line 991, in pandas._libs.parsers.TextReader._read_rows
  File "pandas/_libs/parsers.pyx", line 1123, in pandas._libs.parsers.TextReader._convert_column_data
  File "pandas/_libs/parsers.pyx", line 1169, in pandas._libs.parsers.TextReader._convert_tokens
  File "pandas/_libs/parsers.pyx", line 1215, in pandas._libs.parsers.TextReader._convert_with_dtype
  File "/Users/apple/anaconda3/lib/python3.7/site-packages/pandas/core/dtypes/common.py", line 572, in is_categorical_dtype
    def is_categorical_dtype(arr_or_dtype):
KeyboardInterrupt

During handling of the above exception, another exception occurred:



KeyboardInterrupt



In [ ]:
#cleaning the data from odd symbols, stand alone letters
#lowercasing every document
lentaRu['cleaned'] = lentaRu['text'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^а-яА-Я]",
                                                                                               " ",
                                                                                               str(x)).split() if i not in words]).lower())
#checking out if any cell is NA
lentaRu.isna()

In [ ]:
#getting rid of NAs
lentaRuWon = lentaRu.dropna()
lentaRuWon
print(lentaRuWon['tags'].unique())
print(lentaRuWon['topic'].unique())

In [ ]:
#partitioning the data in to test and train frames
X_train, X_test, y_train, y_test = train_test_split(lentaRuWon['cleaned'],
                                                    lentaRuWon.topic,
                                                    test_size=0.05)

In [ ]:
#creating pipeline
#create TFIDF vector for single words and bigrams
#selecting only good features for our vector
#choosing SVC algorithm
pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2), stop_words=words, sublinear_tf=True)),
                     ('chi',  SelectKBest(chi2, k=10000)),
                     ('clf', LinearSVC(C=1.0, penalty='l1', max_iter=3000, dual=False))])

In [ ]:
#fitting the model into the pipeliine
model = pipeline.fit(X_train, y_train)

In [ ]:
#preparation
#getting instances
vectorizer = model.named_steps['vect']
chi = model.named_steps['chi']
clf = model.named_steps['clf']
#transforming features
feature_names = vectorizer.get_feature_names()
feature_names = [feature_names[i] for i in chi.get_support(indices=True)]
feature_names = np.asarray(feature_names)

In [ ]:
#checking out accuracy score of the model
print("accuracy score: " + str(model.score(X_test, y_test)))
#accuracy score: 0.8038830593120476

In [29]:
import dill as pickle

#with open('testing_model.pk', 'wb') as file:
#    pickle.dump(model, file)
with open('testing_model.pk','rb') as f:
    model = pickle.load(f)


In [36]:
lda_predict = model.predict(top_tokens_vec)
lda_predict = pd.DataFrame(lda_predict)
lda_result = pd.concat([top_tokens_vec, lda_predict], axis=1)
print(lda_result)

                                                   0         0
0  человека жизни лет года сегодня цена год людей...  Из жизни
1  сегодня день лишь стоит гороскоп люди жизни по...  Из жизни
2  жизни день люди каждый людей любовь мир делать...  Из жизни
3  лет грн человека фото хочу каждый доставка мес...       Мир
4  день жизни жизнь сегодня человека год лет скол...  Из жизни
5  жизнь хочу лишь руки стоит жизни года россии л...       Мир
6  жизнь любовь года сегодня люди день году прода...  Из жизни
7  людей люди жизнь жизни любишь хочешь значит ли...  Из жизни
8  руб года лет жизнь день человека тыс мир год ц...  Из жизни
9  жизни люди человека почему поэтому людей кажды...  Из жизни


In [37]:
artm_predict = model.predict(artmToks)
artm_predict = pd.DataFrame(artm_predict)
artm_result = pd.concat([artmToks, artm_predict], axis=1)
print(artm_result)

                                                          0         0
0                                                       NaN  Из жизни
1                                                       NaN  Из жизни
2                                                       NaN  Из жизни
3                                                       NaN  Из жизни
4                                                       NaN  Культура
5                                                       NaN       Мир
6                                                       NaN  Из жизни
7                                                       NaN  Из жизни
8                                                       NaN  Из жизни
9                                                       NaN  Из жизни
10                                                      NaN  Из жизни
11                                                      NaN  Из жизни
12                                                      NaN  Из жизни
13                  